# WCRP Coupled Model Intercomparison Project (CMIP)

CMIP -> Calcular la sensibilidad de clima mediante un ensamble de modelos

The uncertainty associated with the speed of temperature rise is associated with the limitations of our understanding of the global climate system. Each general circulation model used by the IPCC and included in the CMIP5 ensemble uses different assumptions and parameter values to describe the atmospheric changes resulting in growing anthropogenic GHG emissions, and, as a result, the magnitude of the estimated changes varies greatly among different modeling groups. In this respect, one of the features of EDIAM is that it uses 12 GCMs included in the CMIP5 data ensemble to calibrate the parameters


* $\beta$ :  sensibilidad de la atmósfera a emisiones de $CO_2$ (grados celsius)
* $\xi$ : la capacidad de sumidero de carbono de la atmósfera (ppm/BTU/year)
* $\delta$: tasa promedio de regeneración ambiental natural

In [6]:
## Cargamos bibliotecas
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
import math
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
import scipy.stats


In [8]:
#PART1: CALIBRATION OF DELTA TEMP VS LOG CO2 FUNCTION

# Cargamos los datos del csv AllGCMs.csv, los cuales contienen
climate_models = pd.read_csv("/home/milo/PCIC/Maestría/2doSemestre/seminario/scripts/ediam_python/data/ClimateDataCalibration/AllGCMs.csv")

# [degrees Celsius] increase in average global temperature
delta_temp_disater = 7.5  

def linear_reg(datos,modelo,intercepto):
    model_data = datos[datos["Climate.Model"]==modelo]
    n = model_data.shape[0]

    X = model_data['co2.ppm'].apply(math.log).values.reshape(n,1)
    Y = model_data['tas.anomaly'].values.reshape(n,1)

    if intercepto:
        regress = LinearRegression().fit(X,Y)
        R2 = regress.score(X,Y)
        Y_pred= regress.predict(X)
        MSE = mean_squared_error(Y_pred,Y)

        b0 = regress.intercept_[0]
        b1 = regress.coef_[0][0]

        x = model_data['co2.ppm'].apply(math.log).values
        var_b0= (1/n) * ((MSE*np.square(x).sum())/np.square(x -x.mean()).sum())
        var_b1 = MSE/np.square(x -x.mean()).sum()
        t_b0 = b0/math.sqrt(var_b0)
        t_b1 = b1/math.sqrt(var_b1)

        pvalue_b0 = scipy.stats.t.sf(abs(t_b0),n-2)
        pvalue_b1 = scipy.stats.t.sf(abs(t_b1),n-2)

        d = {"Climate.Model": [modelo],
             "Beta.Delta.Temp":[b1],
             "Intercept":[b0],
             "AdjR.squared": [R2],
             "Intercept.P.value": [pvalue_b0],
             "Beta.P.value":  [pvalue_b0]}
        df = pd.DataFrame(data=d)
    else:
        regress = LinearRegression(fit_intercept=False).fit(X,Y)
        df = regress.coef_[0][0]
        

    return df

# Para cada modelo realizamos una regresión lineal entre log(co2.ppm) y tas.anomaly
result = pd.DataFrame()

for model in climate_models["Climate.Model"].unique():
    result = result.append(linear_reg(climate_models,model,True), ignore_index=True)

#estimate CO2 concentrations for delta temp disaster

result['CO2.base'] = (-1*result['Intercept']/result['Beta.Delta.Temp']).apply(math.exp)
result['CO2.Disaster'] =result['CO2.base']*(delta_temp_disater/result['Beta.Delta.Temp']).apply(math.exp)
result['Delta.Temp.Disaster'] = delta_temp_disater
result

,Climate.Model,Beta.Delta.Temp,Intercept,AdjR.squared,Intercept.P.value,Beta.P.value,CO2.base,CO2.Disaster,Delta.Temp.Disaster
0,MRI-ESM1,4.263734,-24.151831,0.979493,6.068348e-207,6.068348e-207,288.437510,1674.885154,7.5
1,MIROC5,4.121571,-23.390097,0.939486,0.000000e+00,0.000000e+00,291.501112,1798.553938,7.5
2,MPI-ESM-LR,4.673517,-26.433039,0.978795,0.000000e+00,0.000000e+00,285.979509,1423.262838,7.5
3,MPI-ESM-MR,4.665210,-26.384382,0.982968,0.000000e+00,0.000000e+00,285.876986,1426.824055,7.5
4,NorESM1-M,4.125460,-23.413881,0.972043,0.000000e+00,0.000000e+00,291.622249,1796.217539,7.5
5,NorESM1-ME,4.336928,-24.544574,0.967822,0.000000e+00,0.000000e+00,286.987093,1617.717931,7.5
6,MIROC-ESM,5.931537,-33.553533,0.973282,0.000000e+00,0.000000e+00,286.232050,1013.566627,7.5
7,MIROC-ESM-CHEM,6.130332,-34.665601,0.972561,0.000000e+00,0.000000e+00,285.650042,970.869657,7.5
8,bcc-csm1-1,4.997053,-28.322606,0.974126,0.000000e+00,0.000000e+00,289.415046,1298.216153,7.5
9,GFDL-ESM2G,3.189799,-18.119386,0.932088,0.000000e+00,0.000000e+00,293.071388,3076.849744,7.5


In [271]:
#PART2: CALIBRATION OF S EQUATION
climate_models = climate_models[['Climate.Model','Year','co2.ppm']].groupby(['Climate.Model','Year'], as_index=False).mean()

#merge CO2 Disaster with raw data
climate_models = climate_models.merge(result[['Climate.Model','CO2.Disaster']], how='inner', on='Climate.Model')


In [325]:
#calculate quality of the environment
climate_models['S'] = climate_models['CO2.Disaster']-climate_models['co2.ppm']

#order data set
climate_models = climate_models.sort_values(by = ['Climate.Model','Year'])

#load data of world consumption of fossil fuels
fossil_fuel = pd.read_csv("/home/milo/PCIC/Maestría/2doSemestre/seminario/scripts/ediam_python/data/ClimateDataCalibration/FossilFuelConsumption.csv")
fossil_fuel = fossil_fuel[["Year","Fossil.Fuels.Consumption"]]

#we make the same assumption as Acemoglous et. al, such that Delta.S=0.5(qsi*Y(t)/S(t-1));
#thus the original model: S(t)=-qsi*Y(t)+(1+Delta.S)*S(t-1) is reduced to: S(t)-S(t-1)=-0.5*qsi*Y(t)
#and can be estimated with lm

s_parameters = pd.DataFrame()

for model in climate_models["Climate.Model"].unique():
    s_parameters = s_parameters.append(compute_delta(climate_models,model), ignore_index=True)

s_parameters

#Create a table with parameters for all climate models
climate_param_ini = result[["Climate.Model","Beta.Delta.Temp","CO2.base","CO2.Disaster","Delta.Temp.Disaster"]]
climate_param = climate_param_ini.merge(s_parameters,how ="inner", on="Climate.Model")
climate_param

,Climate.Model,Beta.Delta.Temp,CO2.base,CO2.Disaster,Delta.Temp.Disaster,qsi,Delta.S,S.0
0,MRI-ESM1,4.263734,288.437510,1674.885154,7.5,0.010046,0.001301,1294.459680
1,MIROC5,4.121571,291.501112,1798.553938,7.5,0.010052,0.001191,1416.700681
2,MPI-ESM-LR,4.673517,285.979509,1423.262838,7.5,0.010073,0.001612,1041.775295
3,MPI-ESM-MR,4.665210,285.876986,1426.824055,7.5,0.010073,0.001606,1045.336614
4,NorESM1-M,4.125460,291.622249,1796.217539,7.5,0.010038,0.001191,1414.661684
5,NorESM1-ME,4.336928,286.987093,1617.717931,7.5,0.010038,0.001359,1236.157718
6,MIROC-ESM,5.931537,286.232050,1013.566627,7.5,0.010039,0.002600,632.612135
7,MIROC-ESM-CHEM,6.130332,285.650042,970.869657,7.5,0.010039,0.002780,589.895814
8,bcc-csm1-1,4.997053,289.415046,1298.216153,7.5,0.010054,0.001823,915.970085
9,GFDL-ESM2G,3.189799,293.071388,3076.849744,7.5,0.010023,0.000630,2695.295397


In [315]:
def compute_delta(datos,modelo):
    datos = datos[datos['Climate.Model']==modelo].reset_index(drop=True)
    datos["S_diff"]= datos["S"].diff() 
    model_data = datos.merge(fossil_fuel, how = "right" ,on = "Year" )    

        
    n = model_data.shape[0]

    X = model_data['Fossil.Fuels.Consumption'].values.reshape(n,1)
    Y = model_data['S_diff'].values.reshape(n,1)
    regress = LinearRegression(fit_intercept=False).fit(X,Y)
    qsi =   regress.coef_[0][0]/-0.5

    #estimate Delta.S

    model_data['S_lag'] = model_data['S']-model_data['S_diff']
    Y = model_data['S_lag'].values.reshape(n,1)
    regress = LinearRegression(fit_intercept=False).fit(X,Y)
    Delta_S = (0.5*qsi)/regress.coef_[0][0]

    #calculate S_hat
    model_data['S_hat'] = (-1*qsi*model_data['Fossil.Fuels.Consumption'])+((1+Delta_S)*model_data['S_lag'])
    
    d = {"Climate.Model" :[modelo],
         "qsi":[qsi],
         "Delta.S":[Delta_S],
         "S.0":model_data[model_data["Year"]==2012]["S_hat"].values}
    df = pd.DataFrame(data=d)
    return df